In [145]:
import numpy as np
from sklearn import preprocessing as pp
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import cross_validation



In [190]:
train_data_raw = pd.read_csv('train_enc.csv', quotechar='"', skipinitialspace=True)
cluster_set = pd.read_csv('variable_cluster.csv', quotechar='"', skipinitialspace=True)

In [191]:
data = train_data_raw.drop(train_data_raw.columns[[0, 2, 6]], axis=1)
data[0:2]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,Unspecified,WIRELESS
0,999,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [59]:
cluster_set[0:2]

,Unnamed: 0,var_name,var_index,cluster
0,1,X1.HR.PHOTO,7,cluster1
1,2,BAKERY,10,cluster1


In [194]:
def label_transform(col):
    le = pp.LabelEncoder()
    le.fit(col)
    label_seq = list(le.classes_)
    col_tr = le.fit_transform(col)
    return (label_seq, col_tr)

cluster = label_transform(cluster_set[['cluster']])[0]

/Library/Python/2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Python/2.7/site-packages/sklearn/preprocessing/label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [51]:
d={}

In [195]:



for clust in cluster:
    inputlist= list(cluster_set[cluster_set['cluster'] == clust]['var_index'])
    inital = 0 
    for index in inputlist:
        inital = inital + data.iloc[:,index]    
    data[clust] = inital
    
    

In [196]:
data[0:5]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,cluster1,cluster10,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9
0,999,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
1,30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,26,1,0,0,0,0,0,0,0,0,...,4,0,0,18,0,0,1,1,4,0
3,8,1,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,0,0,0
4,8,1,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,0


In [197]:

data_iter = data.iloc[:,range(76, 86)]
data_iter[data_iter>0] =1
data_iter[data_iter<0] =-1

/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [133]:
cluster = list(data_iter.columns)
interaction_list = list(pd.MultiIndex.from_product([cluster, cluster], names=['first', 'second']))


In [198]:
#W
cluster = list(data_iter.columns)[0:10]

weekday = list(train_data_raw)[3:10]
interaction_list2=[]
interaction_list2 = list(pd.MultiIndex.from_product([cluster, weekday], names=['first', 'second']))

In [171]:

interaction_list_uni = [tuple(sorted((a,b))) for (a,b) in interaction_list if a!=b ]
len(interaction_list_uni)

2970

In [199]:
#w
interaction_list_uni2 = [tuple(sorted((a,b))) for (a,b) in interaction_list2 if a!=b ]
len(interaction_list_uni2)

70

In [109]:
interaction_list_uni = list(set(interaction_list_uni))


In [200]:
#w
interaction_list_uni2 = list(set(interaction_list_uni2))

In [167]:

len(interaction_list_uni)


45

In [201]:
len(interaction_list_uni2)

70

In [204]:
#w
week = train_data_raw.iloc[:,range(3,10)]
data_x= pd.concat([week, data_iter], axis=1)
data_x_int2=data_x

In [187]:
data_x_int2[0:5]

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,cluster6*cluster7,cluster3*cluster7,cluster2*cluster9,cluster3*cluster9,cluster2*cluster6,cluster1*cluster2,cluster1*cluster9,cluster4*cluster7,cluster5*cluster7,cluster1*cluster5
0,1,0,0,0,0,0,0,-0,-0,0,0,0,0,0,-0,-0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [173]:
data_x_int = data_iter
data_x_int
data_x_int.shape

(95674, 10)

In [130]:
for (a,b) in interaction_list_uni:
    var_name = '*'.join((a,b))
    data_x_int[[var_name]] = data_iter[[a]].multiply(data_iter[b], axis=0)

In [205]:
#w
for (a,b) in interaction_list_uni2:
    var_name = '*'.join((a,b))
    data_x_int2[[var_name]] = data_x[[a]].multiply(data_x[b], axis=0)

In [132]:
data_x_int.shape
data_result = data_x_int.iloc[:,range(10,55)]
data.shape

In [137]:
df1 = data.iloc[:,range(76)]
data_iter_final = pd.concat([df1, data_result], axis=1)

In [217]:
#w

data_result_w = data_x_int2.iloc[:,range(17,87)]
df1 = data.iloc[:,range(76)]
data_iter_final2 = pd.concat([df1, data_result_w], axis=1)
data_iter_final2 = pd.concat([data_iter_final2, data_iter_final.iloc[:,range(75,121)]], axis=1)
data_iter_final2[0:5]

,TripType,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,...,cluster5*cluster8,cluster10*cluster9,cluster1*cluster3,cluster7*cluster9,cluster8*cluster9,cluster4*cluster6,cluster10*cluster5,cluster4*cluster9,cluster7*cluster8,cluster1*cluster6
0,999,1,0,0,0,0,0,0,0,0,...,0,0,0,-0,0,0,0,0,-0,0
1,30,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,26,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
3,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [218]:
data = data_iter_final2

In [219]:
data_x = data.drop('TripType', axis=1)
data_y = data['TripType']

In [220]:
# partition train and test set
n_row = data_x.shape[0]

np.random.seed(0)

row_ind = range(n_row)
np.random.shuffle(row_ind)

train_ind = row_ind[0:int(n_row*0.9)]
test_ind = row_ind[int(n_row*0.9):n_row]

train_x = data_x.iloc[train_ind, :]
train_y = data_y[train_ind]

test_x = data_x.iloc[test_ind, :]
test_y = data_y[test_ind]

In [221]:
# cross validation split
cv = cross_validation.ShuffleSplit(len(train_ind), n_iter=10, test_size=0.1, random_state=0)

In [222]:
# Classification_Tree Classfier
clf = tree.DecisionTreeClassifier()

In [223]:
scores = cross_validation.cross_val_score(clf, train_x, train_y, cv=cv)
np.mean(scores)

0.57038671466728608

In [224]:
clf.fit(train_x, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [225]:
# log loss of test
clf_probs = clf.predict_proba(test_x)

score = log_loss(test_y, clf_probs)

y_pred = clf.predict(test_x)
acc = accuracy_score(test_y, y_pred)

print "Test accuracy =", acc, ", log loss =", score

Test accuracy = 0.575982441472 , log loss = 12.5186316422


In [230]:
# Random Forest Classfier
clf = RandomForestClassifier(n_estimators=150, random_state=0, max_features = "sqrt",class_weight='balanced')

In [231]:
scores = cross_validation.cross_val_score(clf, train_x, train_y, cv=cv)
np.mean(scores)

0.59487864359540121

In [232]:
clf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='sqrt',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [229]:
# log loss of test
clf_probs = clf.predict_proba(test_x)

score = log_loss(test_y, clf_probs)

y_pred = clf.predict(test_x)
acc = accuracy_score(test_y, y_pred)

print "Test accuracy =", acc, ", log loss =", score

Test accuracy = 0.634301839465 , log loss = 1.58041265008
